# Bedrock Stock Agent — Test Notebook

Simple interface to test the deployed agent via Agentcore.

**Setup:** Copy `.env.example` to `.env` and fill in your values before running.

## Imports and Setup

In [ ]:
from dotenv import load_dotenv
import json
import uuid
import urllib.parse
import requests
import boto3
import os

load_dotenv(override=True)

COGNITO_CLIENT_ID = os.getenv("COGNITO_CLIENT_ID")
AWS_REGION = os.getenv("AWS_REGION")
AGENTCORE_RUNTIME_ARN = os.getenv("AGENTCORE_RUNTIME_ARN")
USERNAME = os.getenv("COGNITO_USERNAME")
PASSWORD = os.getenv("COGNITO_PASSWORD")

## Authenticate with Cognito

In [64]:
cognito = boto3.client("cognito-idp", region_name=AWS_REGION)

resp = cognito.initiate_auth(
    ClientId=COGNITO_CLIENT_ID,
    AuthFlow="USER_PASSWORD_AUTH",
    AuthParameters={"USERNAME": USERNAME, "PASSWORD": PASSWORD},
)

access_token = resp["AuthenticationResult"]["IdToken"]
print("Cognito login successful. Access token length:", len(access_token))


/Users/alexandreray/Work/interviews/teilur/bedrock-agentcore-stock-agent/venv/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


Cognito login successful. Access token length: 1098


## Invoke Agent

In [65]:
def invoke_agentcore(prompt: str, user_id: str = USERNAME):
    base_url = f"https://bedrock-agentcore.{AWS_REGION}.amazonaws.com"
    encoded_arn = urllib.parse.quote(AGENTCORE_RUNTIME_ARN, safe="")
    url = f"{base_url}/runtimes/{encoded_arn}/invocations"

    headers = {
        "Content-Type": "application/json",
        "Accept": "text/event-stream",
        "Authorization": f"Bearer {access_token}",
        "X-Amzn-Bedrock-AgentCore-Runtime-User-Id": user_id,
    }

    payload = {"input": {"prompt": prompt}}

    r = requests.post(url, headers=headers, data=json.dumps(payload), stream=True, timeout=300)

    if r.status_code >= 400:
        print("STATUS:", r.status_code)
        print("RESPONSE (first 2000 chars):", r.text[:2000])
        r.raise_for_status()

    lines = []
    for raw in r.iter_lines():
        if not raw:
            continue
        if isinstance(raw, bytes):
            raw = raw.decode("utf-8", errors="replace")
        if raw.startswith("data: "):
            raw = raw[6:]
        lines.append(raw)

    return lines


In [66]:
queries = [
    "What is the stock price for Amazon right now?",
    # "What were the stock prices for Amazon in Q4 last year?",
    # "Compare Amazon's recent stock performance to what analysts predicted in their reports",
    # "I’m researching AMZN give me the current price and any relevant information about their AI business",
    # "What is the total amount of office space Amazon owned in North America in 2024?",
]

for q in queries:
    print("\n" + "=" * 100)
    print("QUERY:", q)
    print("=" * 100)
    print("\n".join(invoke_agentcore(q)))



QUERY: What is the stock price for Amazon right now?
{"type": "agent_reasoning", "content": "[{'type': 'text', 'text': \"I'll help you get the current stock price for Amazon. I'll use the get_realtime_stock_price function to retrieve this information.\"}, {'type': 'tool_use', 'name': 'get_realtime_stock_price', 'input': {'ticker': 'AMZN'}, 'id': 'tooluse_OqW9V5rbu1ov8jXb7dZGJB'}]", "step": 1}
{"type": "tool_call", "tool": "get_realtime_stock_price", "input": {"ticker": "AMZN"}, "step": 1}
{"type": "tool_result", "content": "What is the stock price for Amazon right now?", "step": 2}
{"type": "tool_result", "content": "[{'type': 'text', 'text': \"I'll help you get the current stock price for Amazon. I'll use the get_realtime_stock_price function to retrieve this information.\"}, {'type': 'tool_use', 'name': 'get_realtime_stock_price', 'input': {'ticker': 'AMZN'}, 'id': 'tooluse_OqW9V5rbu1ov8jXb7dZGJB'}]", "step": 2}
{"type": "tool_result", "content": "{\n  \"ticker\": \"AMZN\",\n  \"cur